### Quotes

In [1]:
import pandas as pd

# Read the CSVs
df_benchmark = pd.read_csv('daily_benchmark_quotes.csv', index_col=0)
df_stock = pd.read_csv('daily_stock_quotes.csv', index_col=0)
df_bond = pd.read_csv('daily_bond_quotes.csv', index_col=0)

# Merge the dataframes
df = df_benchmark.copy()
df = df.merge(df_stock, on='Date', how='left')  # Default suffixes=('_x','_y')
df = df.merge(df_bond, on='Date', how='left')   # More suffixes if same cols exist

# Function to clean column names
def clean_column_names(dataframe):
    # Find all columns with _x suffix
    x_cols = [col for col in dataframe.columns if col.endswith('_x')]
    
    # For each _x column, drop the corresponding _y column and rename _x
    for x_col in x_cols:
        base_name = x_col[:-2]  # Remove _x suffix
        y_col = f"{base_name}_y"
        
        # If y_col exists, drop it and rename x_col
        if y_col in dataframe.columns:
            dataframe = dataframe.drop(columns=[y_col])
            dataframe = dataframe.rename(columns={x_col: base_name})
    
    return dataframe

# Apply the function
df = clean_column_names(df).sort_index(axis=0, ascending=True)

# Drop the last 2 rows
df = df.iloc[:-2]

# Save to CSV
daily_output = 'daily_quotes.csv'
monthly_output = 'monthly_quotes.csv'
df.sort_index(axis=0, ascending=True).sort_index(axis=1, ascending=True).to_csv(daily_output)

df_monthly = (df.set_index(pd.to_datetime(df.pop('Date')))
      if 'Date' in df.columns else df.copy())

df_monthly.index = pd.to_datetime(df.index)              
(df_monthly.resample('MS').last()
   .reset_index()
   .to_csv(monthly_output, index=False))